<a href="https://colab.research.google.com/github/jaredguting/colab-projects/blob/main/SampleForecastingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dir_name = '/content/drive/MyDrive/Colab Notebooks/Datasets/'

Mounted at /content/drive


###Connect to Google Sheets

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
try:
    sh = gc.create('Forecasted Products Quantity')  # Will not run if the sheet already exists so make sure to delete existing
    worksheet = sh.get_worksheet(0)
except Exception as e:
    sh = gc.open('Forecasted Products Quantity')
    worksheet = sh.get_worksheet(0)

###Importing Dataset

In [ ]:
import pandas as pd
file_path = '/content/drive/My Drive/Colab Notebooks/Datasets/Generated_Bakeshop.csv'
df = pd.read_csv(file_path)

###Prophet Installation

In [ ]:
!pip install prophet
from prophet import Prophet

###Creating Holiday Parameter

In [ ]:
holidays = pd.DataFrame({
    'holiday': 'Valentine\'s Day',
    'ds': pd.to_datetime(['2024-02-14', '2025-02-14'])
})

holidays = holidays.append(pd.DataFrame({
    'holiday': 'Father\'s Day',
    'ds': pd.to_datetime(['2024-06-16', '2025-06-15'])
}), ignore_index=True)

holidays = holidays.append(pd.DataFrame({
    'holiday': 'Mother\'s Day',
    'ds': pd.to_datetime(['2024-05-12', '2025-05-11'])
}), ignore_index=True)

holidays = holidays.append(pd.DataFrame({
    'holiday': 'Grandparents Day',
    'ds': pd.to_datetime(['2024-09-08', '2025-09-07'])
}), ignore_index=True)

holidays = holidays.append(pd.DataFrame({
    'holiday': 'New Year\'s Eve',
    'ds': pd.to_datetime(['2023-12-31', '2024-12-31'])
}), ignore_index=True)

holidays = holidays.append(pd.DataFrame({
    'holiday': 'Christmas Eve',
    'ds': pd.to_datetime(['2023-12-24', '2024-12-24'])
}), ignore_index=True)

holidays = holidays.append(pd.DataFrame({
    'holiday': 'Teacher\'s Day',
    'ds': pd.to_datetime(['2024-10-05'])
}), ignore_index=True)

<ipython-input-6-3b2c4c9e91f6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  holidays = holidays.append(pd.DataFrame({
<ipython-input-6-3b2c4c9e91f6>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  holidays = holidays.append(pd.DataFrame({
<ipython-input-6-3b2c4c9e91f6>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  holidays = holidays.append(pd.DataFrame({
<ipython-input-6-3b2c4c9e91f6>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  holidays = holidays.append(pd.DataFrame({
<ipython-input-6-3b2c4c9e91f6>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.c

In [ ]:
all_forecasts = []

for product in df['PRODUCT'].unique():
    product_df = df[df['PRODUCT'] == product].copy()
    product_df.rename(columns={'Date': 'ds', 'Qty': 'y'}, inplace=True)
    product_df['ds'] = pd.to_datetime(product_df['ds'])
    product_df.sort_values('ds', inplace=True)

    model = Prophet(daily_seasonality=True, yearly_seasonality=True, holidays=holidays)
    model.fit(product_df)

    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)

    # Preparing forecast data for export
    forecast['Product'] = product
    forecast_future_only = forecast[['Product', 'ds', 'yhat']].copy()
    forecast_future_only['yhat'] = forecast_future_only['yhat'].round()
    forecast_future_only.rename(columns={'yhat': 'Quantity', 'ds': 'Date'}, inplace=True)

    all_forecasts.append(forecast_future_only)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpmlrb71_g/umsd0rze.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmlrb71_g/893wdnki.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=33264', 'data', 'file=/tmp/tmpmlrb71_g/umsd0rze.json', 'init=/tmp/tmpmlrb71_g/893wdnki.json', 'output', 'file=/tmp/tmpmlrb71_g/prophet_modeldji45hfh/prophet_model-20240207065104.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
06:51:04 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
06:51:04 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmlrb71_g/1szvwyr9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmlrb71_g/zskxxkvx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [ ]:
forecast_export = pd.concat(all_forecasts)

forecast_export['Date'] = pd.to_datetime(forecast_export['Date'])

forecast_export['Date'] = forecast_export['Date'].dt.strftime('%Y-%m-%d')

values = [forecast_export.columns.values.tolist()] + forecast_export.values.tolist()

worksheet.update('A1', values)

{'spreadsheetId': '1FIv1c8EBGAfeb50MrgwZLt6DyLYMUrxlX2t1waRSpHU',
 'updatedRange': 'Sheet1!A1:C77725',
 'updatedRows': 77725,
 'updatedColumns': 3,
 'updatedCells': 233175}